### Convert Text To Audio

In [1]:
#!pip install --upgrade pip

In [2]:
#!pip install gTTS

In [3]:
#!pip install playsound

In [4]:
#!pip install pygobject

#### Multi Text To Speech

In [5]:
import os
import numpy as np
import pandas as pd
from gtts import gTTS
from playsound import playsound
from pathlib import Path
import shutil
import time

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


In [6]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# lang
if lang_folder == "English":
    lang = 'en'
elif lang_folder == "Turkish":
    lang = 'tr'
elif lang_folder == "German":
    lang = 'de'
elif lang_folder == "French":
    lang = 'fr'
elif lang_folder == "Spanish":
    lang = 'es'
elif lang_folder == "Arabic":
    lang = 'ar'
elif lang_folder == "Portuguese":
    lang = 'pt'
elif lang_folder == "Dutch":
    lang = 'nl'
elif lang_folder == "Italian":
    lang = 'it'
else:
    pass

# word start and end
word_start = 0  # 0  # native word start index
word_end = 200  # 200  # native word end index

print(f"{lang_folder}\n{lang}") 

Turkish
tr


In [7]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/200 Words"  
# f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"

Path(path).mkdir(parents=True, exist_ok=True)

In [8]:
def text_convert_to_audio(text_list, out_folder_name, out_path, language="en", speech_slow=True):
    ''' 
    text_convert_to_audio(text_list, out_folder_name, out_path, language="en", speech_slow=True)\n
    text_list is list values. out_folder_name and out_path are path in str\n
    ex:\n
    path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    text_convert_to_audio(word_select_list, "Text_Audio", path, language=lang)
    '''
    result_path = f"{out_path}/{out_folder_name}"
    Path(result_path).mkdir(parents=True, exist_ok=True)
    for text in text_list:
        text_var = f"{text}"
        textaudio = gTTS(text=text_var, lang=language, slow=speech_slow)
        textaudio.save(f"{result_path}/{text}.mp3")
        time.sleep(0.2) # sleep 0.2 second

In [9]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
#df_word_all = pd.read_excel("test.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987922,karneleme,5
987923,karnaya,5
987924,dörtlümüzün,5
987925,karnavalınız,5


In [10]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [11]:
word_select_list = df_word_select["word"].values.tolist()

In [12]:
text_convert_to_audio(word_select_list, "Text_Audio", path, language=lang)

#### Multi Audio Convert Same Codec

In [13]:
from moviepy.editor import *
import pandas as pd
import numpy as np
import os
import sys
from os.path import isfile, join
import glob
from pathlib import Path
import shutil
import time

In [14]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner

# word start and end
#word_start = 0  # 0  # native word start index
#word_end = 200  # 200  # native word end index

In [15]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/200 Words"  
# f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"

Path(path).mkdir(parents=True, exist_ok=True)

In [16]:
audio_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/200 Words"
# f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"

In [17]:
def convert_audio_to_mp3(data_folder_name, data_path, out_folder_name, out_path):
    '''
    convert_audio_to_mp3(data_folder_name, data_path, out_folder_name, out_path):\n
    data_folder_name, data_path, out_folder_name and out_path are path and folder in string.\n
    ex.\n
    path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    audio_path = f"/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Image Audio/Product"\n
    convert_audio_to_mp3("Text_Audio", audio_path, "Text_Audio_MP3", path)
    '''
    result_path = f"{out_path}/{out_folder_name}"
    data_folder_path = f"{data_path}/{data_folder_name}"
    Path(result_path).mkdir(parents=True, exist_ok=True)
    files=[f for f in os.listdir(data_folder_path) if isfile(join(data_folder_path,f))]    
    for i in range(len(files)):
        #text = f"{text}"
        pathfilename=data_folder_path+'/'+files[i]
        filename_without_ext = os.path.splitext(files[i])[0]
        try:
            audioclip = AudioFileClip(pathfilename)
            audioclip.write_audiofile(f"{result_path}/{filename_without_ext}.mp3", codec="mp3", bitrate="128k", fps=44100)
            time.sleep(0.1)
        except:
            print("Oops!", sys.exc_info()[0], "occurred.")

In [18]:
convert_audio_to_mp3("Text_Audio", audio_path, "Text_Audio_MP3", path)

MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/200 Words/Text_Audio_MP3/bir.mp3


MoviePy - Done.


MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/200 Words/Text_Audio_MP3/bu.mp3


MoviePy - Done.


MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/200 Words/Text_Audio_MP3/ne.mp3


...

MoviePy - Done.


MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/200 Words/Text_Audio_MP3/neler.mp3


MoviePy - Done.


MoviePy - Writing audio in /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Data/200 Words/Text_Audio_MP3/ister.mp3


MoviePy - Done.


#### Text Audio Concatenate

In [ ]:
from moviepy.editor import AudioFileClip, concatenate_audioclips
import pandas as pd
import numpy as np
import glob
import os
from pathlib import Path
import shutil
import time

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# word start and end
word_start = 0  # 0  # native word start index
word_end = 20  # 200  # native word end index

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize}/Image Audio Video/Data/200 Words/Audio_Concat_MP3"
Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
audio_path_1 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize}/Image Audio Video/Data/200 Words/Text_Audio_MP3"
audio_path_2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize}/Image Audio Video/Data/200 Words/Audio_Forvo"

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

In [ ]:
word_select_list = df_word_select["word"].values.tolist()

In [ ]:
no_audio_file_list = []
for word in word_select_list:
    audio_file_path_var1 = glob.glob(f"{audio_path_1}/{word}.mp3")
    audio_file_path_var2 = glob.glob(f"{audio_path_2}/{word}.mp3")
    audio_file_path_list = audio_file_path_var1 + audio_file_path_var2
    audio_list = []
    for audio_file_path in audio_file_path_list:
        audio_var = AudioFileClip(f"{audio_file_path}")
        audio_list.append(audio_var)
    try:
        audio_combined = concatenate_audioclips(audio_list)
        audio_combined.write_audiofile(f"{path}/{word}.mp3")
        time.sleep(0.2)  # optional
    except:
        no_audio_file_list.append(word)
        pass
    
no_audio_file_list

In [ ]:
no_audio_file_list

#### Sample

In [ ]:
import os
import numpy as np
import pandas as pd
from gtts import gTTS
from playsound import playsound

In [ ]:
# The text that you want to convert to audio
mytext = 'Welcome to geeksforgeeks!'

In [ ]:
# Language in which you want to convert
language = 'en'

In [ ]:
# Passing the text and language to the engine, 
# here we have marked slow=False. Which tells 
# the module that the converted audio should 
# have a high speed
myobj = gTTS(text=mytext, lang=language, slow=False)

In [ ]:
# Saving the converted audio in a mp3 file named
# welcome 
myobj.save("welcome.mp3")

In [ ]:
# Playing the converted file
os.system("mpg321 welcome.mp3")